```
Author: Yagnik Poshiya
GitHub: @yagnikposhiya
U & PU. Patel Department of Computer Engineering,
Chandubhai S. Patel Institute of Technology,
Charotar University of Science and Technology,
Changa-388421, Anand, Gujarat, India.
```

# Implementation of Convolution Neural Network

## Load Libraries

In [ ]:
import tensorflow
import matplotlib
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, GlobalAveragePooling2D, Dense, MaxPool2D
from keras import Model, Input
from keras.optimizers import Adam
from keras.metrics import CategoricalAccuracy
from keras.utils import plot_model

## Load Dataset

In [ ]:
def transform_one(ORG_IMG):
    return cv2.flip(ORG_IMG,1)

def transform_two(ORG_IMG):
    return cv2.flip(ORG_IMG,0)

def transform_three(ORG_IMG):
    return cv2.rotate(ORG_IMG, cv2.ROTATE_180)


def preprocessingFunction(IMG):
    TRANSFORM_FLIP_1 = transform_one(IMG)
    TRANSFORM_FLIP_0 = transform_two(TRANSFORM_FLIP_1)
    ROTATE_180_DEGREE = transform_three(TRANSFORM_FLIP_0)
    RESIZED_IMG = cv2.resize(ROTATE_180_DEGREE, (224,224))
    return RESIZED_IMG

In [ ]:
datagen_train = ImageDataGenerator(samplewise_center=True,
                                   rescale=1./255,
                                   fill_mode='nearest',
                                   brightness_range=[0.4,1.5],
                                   rotation_range=10,
                                   zoom_range=0.1,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   preprocessing_function=preprocessingFunction) # generate the object for the ImageDataGenerator class for training set
datagen_valid = ImageDataGenerator(samplewise_center=True) # generate the object for the ImageDataGenerator class for validation set

train_set = datagen_train.flow_from_directory('/path_to_train_directory/train/',
                                              target_size=(224,224),
                                              color_mode="rgb",
                                              class_mode="categorical",
                                              batch_size=32,
                                              save_to_dir='/path_to_augmented_directory/_augmented_/',
                                              save_prefix='aug',
                                              save_format='png') # "class_mode" is "categorical" because here the problem is multi-class problem.
validation_set = datagen_valid.flow_from_directory('/path_to_validation_directory/validation/',
                                                   target_size=(224,224),
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   batch_size=32)

print("Classes and their Indices: \n{a}".format(a=train_set.class_indices)) # printing the classes with their numerical labels given by ImageDataGenerator class

In [ ]:
def showDataDistribution():
    """
    Extract the exact number of data samples in various sets,
    are used for the model training, evaluation, and testing.
    """
    plt.bar(['Train','Validation'],
            [train_set.samples, validation_set.samples],
            align='center',
            color=['green','red']) # setting alignment of the text to the center
                                            # here x will be ['Train','Validation','Test']
                                            # y will be [train_set.samples, validation_set.samples, test_set.samples]
                                            # colors of respective bars will be ['green','red','blue']
    plt.ylabel('Number of Images') # setting the y-axis label
    plt.title('Data Distribution') # giving title to the bar plot
    for iteration in range(2): # putting tag of total number of data samples available in the various sets,
                                # on the top of the bar with bounding box.
        x = ['Train','Validation']
        y = [train_set.samples,validation_set.samples]
        plt.text(x=x[iteration],
                 y=y[iteration],
                 s=str(y[iteration]),
                 ha='center', # setting horizontal-alignment to the 'center'
                 bbox=dict(facecolor='yellow',alpha=0.8)) # here alpha is unknown, also mentioned in the documentation
    plt.show()

showDataDistribution() # calling the function

## Neptune Introduction

In [ ]:
import neptune

run = neptune.init_run(project='<your_neptune_username>/<project_name>',
                       api_token="<api_token>",
                       source_files=['live_fruit_classification_without_preprocessing.ipynb'])

In [ ]:
import neptune

project = neptune.init_project(project='<your_neptune_username>/<project_name>',
                               api_token="<api_token>")

project['dataset/validation'].track_files('/path_to_validation_directory/validation/')

In [ ]:
import neptune

model = neptune.init_model(project='<your_neptune_username>/<project_name>',
                           api_token='<api_token>',
                           name='CNN-1',
                           key='MO1')


In [ ]:
model_version = neptune.init_model_version(project='<your_neptune_username>/<project_name>',
                                           api_token='<api_token>',
                                           model='DLW-MO1')


In [ ]:
model_version['model_1/h5'].upload('/path_to_model_directory/_model_/fruit_classification_without_preprocessing.h5')

In [ ]:
params = {'batch_size':32,
          'learning_rate':0.001,
          'epochs':6,
          'steps_per_epoch':train_set.samples/train_set.batch_size,
          'validation_steps':validation_set.samples/validation_set.batch_size,
          'loss':'categorical_crossentropy',
          'metrics':'accuracy'}

run['training/model/parameters'] = params

In [ ]:
from neptune.integrations.tensorflow_keras import NeptuneCallback
neptune_cbk = NeptuneCallback(run=run,
                              base_namespace='train_metadata')

## Model Architecture

In [ ]:
IMG_SIZE = 224 # setting the image dimension to the 224
ker_init = "he_normal" # initializing the kernel initializer using the "he_normal" method because "relu" activation is used

input_layer = Input(shape=(IMG_SIZE,IMG_SIZE,3)) # defining the global input layer
"""
Here every where padding value is "same", it means the is half padded,
and this type of padding is called SAME type because output size is same
as the input size when stride=1.
"""
conv1 = Conv2D(64, 3, activation='relu', kernel_initializer=ker_init, padding='same')(input_layer)
conv2 = Conv2D(64, 3, activation='relu', kernel_initializer=ker_init, padding='same')(conv1)
maxpool1 = MaxPool2D((2,2))(conv2)
conv3 = Conv2D(128, 3, activation='relu', kernel_initializer=ker_init, padding='same')(maxpool1)
conv4 = Conv2D(128, 3, activation='relu', kernel_initializer=ker_init, padding='same')(conv3)
maxpool2 = MaxPool2D((2,2))(conv4)
conv5 = Conv2D(256, 3, activation='relu', kernel_initializer=ker_init, padding='same')(maxpool2)
conv6 = Conv2D(256, 3, activation='relu', kernel_initializer=ker_init, padding='same')(conv5)
maxpool3 = MaxPool2D((2,2))(conv6)
averagePooling = GlobalAveragePooling2D()(maxpool3)
dense1 = Dense(64, activation='relu')(averagePooling)
dense2 = Dense(64, activation='relu')(dense1)
dense3 = Dense(32, activation='relu')(dense2)
output_layer = Dense(6, activation='softmax')(dense3) # defining the global output layer with "softmax" activation

model = Model(input_layer, output_layer) # defining the sequential model using Model class

model.compile(loss=params['loss'], # using "categorical_crossentropy" due to multi-class problem
              optimizer=Adam(learning_rate=params['learning_rate']), # setting learning rate to the 0.001, min(learning_rate) = high(performance)
              metrics = CategoricalAccuracy(name=params['metrics'])) # using alias "accuracy" of CategoricalAccuracy

In [ ]:
model.summary() # getting alias of the all layers with their parameters and total number of trainable and non-trainable parameters

## Plot the Architecture

In [ ]:
plot_model(model,
           show_shapes=True, # showing the input and output shapes
           show_dtype=True, # showing data types of layer
           show_layer_names=True, # showing the name of the layer
           rankdir='TB', # plot the graph in vertical manner (if 'LR' is passed instead of 'TB' then graph will plotted in horizontal manner)
           expand_nested=True, # showing layers of cluster separately
           show_layer_activations=True, # showing the activation function of the layer
           dpi=128) # printing dots_per_inch {high value = high resolution}

## Plot the 3D Architecture

In [ ]:
import visualkeras
from PIL import ImageFont
"""
Here defaultdict is used because if a key is not found in the dictionary,
then instead of a KeyError being thrown, a new entry is created.
"""
from collections import defaultdict

font = ImageFont.truetype(font="/path_to_font_file/arial.ttf", size=32)
color_map = defaultdict(dict)
color_map[Conv2D]['fill'] = 'orange'
color_map[Dense]['fill'] = 'green'

visualkeras.layered_view(model, legend=True, spacing=25, color_map=color_map, font=font, to_file='output_1.png').show()
# show() is used to show the output on screen

## Train the Model

In [ ]:
history = model.fit(train_set,
                    validation_data=validation_set,
                    steps_per_epoch=params['steps_per_epoch'],
                    validation_steps=params['validation_steps'],
                    epochs=params['epochs'],
                    batch_size=params['batch_size'],
                    callbacks=neptune_cbk) # setting the epochs to 6, means all batches are passed through network architecture upto 6 times

In [ ]:
model.summary()

In [ ]:
model.save('/path_to_model_directory/_model_/fruit_classification_without_preprocessing.h5')
# saving the model on the given location

## Plot the Learning Curves

In [ ]:
values = history.history # getting history of the performance of the model during training time

training_accuracy=values['accuracy'] # training accuracy
validation_accuracy=values['val_accuracy'] # validation accuracy

training_loss=values['loss'] # training loss
validation_loss=values['val_loss'] # validation loss

epoch = range(len(training_accuracy)) # generating a list with length equal to the length of the "training_accuracy" list

In [ ]:
plt.figure()
# plotting the graph epoch vs training accuracy, epoch vs validation accuracy
plt.plot(epoch,
         training_accuracy,
         label='training accuracy')
plt.plot(epoch,
         validation_accuracy,
         label='validation acuuracy')
plt.title('Training and validation accuracy') # giving the title to the plot
plt.xlabel('epoch') # setting the xlabel
plt.ylabel('accuracy') # setting the ylabel
plt.legend()

plt.savefig('/path_to_figures_directory/_figures_/training-validation_accuracy.png',
            format='png',
            dpi=400) # saving the figure with high dots_per_inch in "png" format

In [ ]:
plt.figure()
# plotting the graph epoch vs training accuracy, epoch vs validation loss
plt.plot(epoch,
         training_loss,
         label='training loss')
plt.plot(epoch,
         validation_loss,
         label='validation loss')
plt.title('Traning and validation loss') # giving the title to the plot
plt.xlabel('epoch') # setting the xlabel
plt.ylabel('loss') # setting the ylabel
plt.legend()

plt.savefig('/path_to_figures_directory/_figures_/training-validation_loss.png',
            format='png',
            dpi=400) # saving the figure with high dots_per_inch in "png" format

## Test the Model

In [ ]:
"""
Testing the model on the single image
OR
single input.
"""
IMG = cv2.imread('/path_to_test_directory/test/fb2.png') # reading an image from the given path in rgb mode
RESIZE_IMG = cv2.resize(IMG, (224,224)) # resizing the image to the shape (224,224)
PROCESSED_IMG = np.array(RESIZE_IMG).reshape(-1,224,224,3) # reshaping the image to the (-1,224,224,3) due to single image

In [ ]:
prediction_prob = model.predict(PROCESSED_IMG) # giving processed image to the model 
print("Classwise probability: {a}".format(a=prediction_prob))
print("Classes: {b}".format(b=train_set.class_indices))

In [ ]:
run.stop()

In [ ]:
model.stop()
model_version.stop()

In [ ]:
project.stop()